# Logistic Regression

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.spatial
from scipy.spatial.distance import euclidean
from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib.pyplot as plt
import statistics
import sys
import math
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Data = pd.read_csv("BSOM_DataSet_for_HW2.csv")

In [2]:
X = Data.loc[:,['all_mcqs_avg_n20','all_NBME_avg_n4']].values
y= Data.loc[:,['LEVEL']].values
#Xandy = Data.loc[:,['all_mcqs_avg_n20','all_NBME_avg_n4','LEVEL']]



train_X, test_X = X[:74,:], X[74:,:]
train_y, test_y = y[:74,:], y[74:,:]



#train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 1/3, random_state = 0)
#N,X_Feat = X_train.shape

train_N, train_X_Features = train_X.shape

# Question 3a Logistic Regression with Multiple variable

In [4]:
def sigmoid(z):
    g_of_z= 1/(1+np.exp(-z))
    return g_of_z

def iteration(y_temp,DataXX,theta):
    eta = 0.2
    prev_theta = 0
    for j in range(10000):
        output = DataXX.dot(theta)
        h = sigmoid(output)
        errors =  sigmoid(output) - y_temp
        cost_Fn = (-1/train_N)*np.sum((y_temp*np.log(h))+((1-y_temp)*np.log(1-h)))
        prev_theta = theta
        derivative = (1/train_N)* DataXX.T.dot(errors)
        theta  = theta - (eta * derivative)
        if(np.allclose(prev_theta,theta)):
            print("break theta:",theta)
            break
    #hypothesis = sigmoid(DataXX.dot(theta))
    return theta

In [36]:

DataXX = np.insert(train_X,0,1,axis=1)
#np.random.seed(233)
ttheta =np.random.randint(100,size= (train_X_Features+1, 1))

iter=1000


#A and not B,C,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 1,'B': 0,'C':0,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
#print("y_copy 1:",y_temp)
#print("y_copy shape:",y_temp.shape)
theta_ovrA= iteration(y_temp,DataXX,ttheta)
hA=sigmoid(DataXX.dot(theta_ovrA))
#print("hypothesis",hA)

#B and not A,C,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 1,'C':0,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrB=  iteration(y_temp,DataXX,ttheta)
hB=sigmoid(DataXX.dot(theta_ovrB))
#print("hypoB:",hB)

#C and not A,B,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 0,'C':1,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrC=  iteration(y_temp,DataXX,ttheta)
hC=sigmoid(DataXX.dot(theta_ovrC))
#print("hypoC:",hC)

#D and not A,B,C
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 0,'C':0,'D':1} 
y_copy= [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrD=  iteration(y_temp,DataXX,ttheta)
hD=sigmoid(DataXX.dot(theta_ovrD))
#print("hypoD:",hD)

concat_hypo  = np.hstack([hA, hB, hC,hD])
#print("hypo final:",concat_hypo )
#Return index of max values
hypo_final = np.argmax(concat_hypo,axis =1)
print("hypo final:",hypo_final )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


break theta: [[  6.93013304]
 [ 24.03197047]
 [-36.19724331]]
hypo final: [2 2 1 0 0 2 2 2 2 0 0 1 0 1 0 2 1 0 2 1 1 0 0 0 0 0 0 0 2 2 0 2 0 0 0 2 0
 1 1 0 2 1 1 0 2 0 1 2 2 0 2 1 0 2 2 1 0 0 2 0 2 0 0 2 2 0 1 2 2 0 0 0 0 1]


In [6]:
testDataXX = np.insert(test_X,0,1,axis=1)
test_hA=sigmoid(testDataXX.dot(theta_ovrA))
test_hB=sigmoid(testDataXX.dot(theta_ovrB))
test_hC=sigmoid(testDataXX.dot(theta_ovrC))
test_hD=sigmoid(testDataXX.dot(theta_ovrD))
concat_hypo  = np.hstack([test_hA, test_hB, test_hC,test_hD])

#Return index of max values
y_pred = np.argmax(concat_hypo,axis =1)
print("hypo final test:",y_pred )

hypo final test: [2 0 2 0 0 1 1 1 0 1 1 2 2 0 1 1 0 0 1 2 0 0 2 0 1 0 0 0 1 1 1 2 1 2 1 0 2
 0]


# Question 3b Metrics 

In [7]:
y = Data.loc[:,['LEVEL']]
level = {'A': 0,'B': 1,'C':2,'D':3} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_train = y_copy[:74,None]
y_test =  y_copy[74:,None]

In [8]:

def confu_mat(y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))

In [9]:

def recall(y_test, y_pred):
    recall_score(y_test, y_pred, average='macro')  
def precision(y_test, y_pred):
    precision_score(y_test, y_pred, average='macro') 

In [10]:
def fscore(y_true, y_pred):
    f1_macro = f1_score(y_test, y_pred, average='macro')  
    print("F1 macro :" ,f1_macro)
    f1_micro  = f1_score(y_test, y_pred, average='micro')  
    print("F1 micro :" ,f1_micro)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')  
    print("F1 weighted :" ,f1_weighted)
    f1 = f1_score(y_test, y_pred, average=None)
    print("F1 score :" ,f1)      
          
          
          

In [11]:
confu_mat(y_test, y_pred)
precision(y_test, y_pred)
recall(y_test, y_pred)
fscore(y_test, y_pred)

[[ 7  0  0  0]
 [ 8 10  2  0]
 [ 0  3  6  0]
 [ 0  1  1  0]]
F1 macro : 0.4728163992869875
F1 micro : 0.6052631578947368
F1 weighted : 0.5847171404446945
F1 score : [0.63636364 0.58823529 0.66666667 0.        ]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:

print(classification_report(y_test, y_pred, target_names=['A', 'B', 'C','D']))
print("Accuracy:",accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          A       0.47      1.00      0.64         7
          B       0.71      0.50      0.59        20
          C       0.67      0.67      0.67         9
          D       0.00      0.00      0.00         2

avg / total       0.62      0.61      0.58        38

Accuracy: 0.6052631578947368


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Question 4a Feature scaling 

In [14]:
mean_norm_X = train_X-np.mean(train_X)/(np.max(train_X)-np.min(train_X))
DataXX = np.insert(mean_norm_X,0,1,axis=1)
#np.random.seed(233)
ttheta =np.random.randint(100,size= (train_X_Features+1, 1))

iter=1000


#A and not B,C,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 1,'B': 0,'C':0,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
#print("y_copy 1:",y_temp)
#print("y_copy shape:",y_temp.shape)
theta_ovrA= iteration(y_temp,DataXX,ttheta)
hA=sigmoid(DataXX.dot(theta_ovrA))
#print("hypothesis",hA)

#B and not A,C,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 1,'C':0,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrB=  iteration(y_temp,DataXX,ttheta)
hB=sigmoid(DataXX.dot(theta_ovrB))
#print("hypoB:",hB)

#C and not A,B,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 0,'C':1,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrC=  iteration(y_temp,DataXX,ttheta)
hC=sigmoid(DataXX.dot(theta_ovrC))
#print("hypoC:",hC)

#D and not A,B,C
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 0,'C':0,'D':1} 
y_copy= [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrD=  iteration(y_temp,DataXX,ttheta)
hD=sigmoid(DataXX.dot(theta_ovrD))
#print("hypoD:",hD)

concat_hypo  = np.hstack([hA, hB, hC,hD])

#Return index of max values
hypo_final = np.argmax(concat_hypo,axis =1)
print("hypo final:",hypo_final )

mean_norm_testX = test_X-np.mean(test_X)/(np.max(test_X)-np.min(test_X))
testDataXX = np.insert(mean_norm_testX,0,1,axis=1)
test_hA=sigmoid(testDataXX.dot(theta_ovrA))
test_hB=sigmoid(testDataXX.dot(theta_ovrB))
test_hC=sigmoid(testDataXX.dot(theta_ovrC))
test_hD=sigmoid(testDataXX.dot(theta_ovrD))
concat_hypo  = np.hstack([test_hA, test_hB, test_hC,test_hD])

#Return index of max values
y_pred = np.argmax(concat_hypo,axis =1)
print("hypo final test:",y_pred )

hypo final: [2 1 1 0 0 1 2 1 2 1 1 1 0 1 0 2 1 1 2 1 1 1 1 0 1 0 0 0 2 2 1 2 0 0 1 2 1
 1 1 0 2 1 1 0 2 0 1 2 1 1 2 1 0 2 2 1 0 0 1 0 2 0 0 2 2 1 1 2 2 0 1 0 0 1]
hypo final test: [1 0 1 0 0 1 1 1 0 1 1 1 2 0 1 1 0 0 1 2 0 0 2 0 1 0 0 0 1 1 1 2 1 2 1 0 2
 0]


In [15]:
confu_mat(y_test, y_pred)
precision(y_test, y_pred)
recall(y_test, y_pred)
fscore(y_test, y_pred)

[[ 7  0  0  0]
 [ 8 11  1  0]
 [ 0  5  4  0]
 [ 0  1  1  0]]
F1 macro : 0.44107289107289105
F1 micro : 0.5789473684210527
F1 weighted : 0.5564851933272986
F1 score : [0.63636364 0.59459459 0.53333333 0.        ]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
print(classification_report(y_test, y_pred, target_names=['A', 'B', 'C','D']))
print("Accuracy:",accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          A       0.47      1.00      0.64         7
          B       0.65      0.55      0.59        20
          C       0.67      0.44      0.53         9
          D       0.00      0.00      0.00         2

avg / total       0.58      0.58      0.56        38

Accuracy: 0.5789473684210527


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Question 4 b Regularization 

In [34]:
def sigmoid(z):
    g_of_z= 1/(1+np.exp(-z))
    return g_of_z

def iterationr(y_temp,DataXX,theta,lamda):
    eta = 0.2
    prev_theta = 0
    
    for j in range(10000):
        output = DataXX.dot(theta)
        h = sigmoid(output)
        
        cost_Fn = (-1/train_N)*np.sum((y_temp*np.log(h))+((1-y_temp)*np.log(1-h)))+ lamda/(2*train_N) * sum(np.linalg.norm(theta,1,axis=0))
        prev_theta = theta
        derivative =  DataXX.T.dot(h-y_temp)
        theta  = theta - (eta/train_N * derivative + theta*(lamda/train_N))
        theta[0] = theta[0] - (eta/train_N)* DataXX.T.dot(h - y_temp)[0]
        
        #print("theta:",  theta[0])
        if(np.allclose(prev_theta,theta)):
            print("break theta:",theta,j)
            break
    #hypothesis = DataXX.dot(theta)
    return theta

In [35]:



DataXX = np.insert(train_X,0,1,axis=1)
#np.random.seed(233)
ttheta =np.random.randint(100,size= (train_X_Features+1, 1))


lamda = [0.0001,0.001,0.01,0.1,0.2,1,10]
for l in lamda:
    #A and not B,C,D
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 1,'B': 0,'C':0,'D':0} 
    y_copy = [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    #print("y_copy 1:",y_temp)
    #print("y_copy shape:",y_temp.shape)
    theta_ovrA= iterationr(y_temp,DataXX,ttheta,l)
    hA=sigmoid(DataXX.dot(theta_ovrA))
    #print("hypothesis",hA)

    #B and not A,C,D
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 0,'B': 1,'C':0,'D':0} 
    y_copy = [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    theta_ovrB=  iterationr(y_temp,DataXX,ttheta,l)
    hB=sigmoid(DataXX.dot(theta_ovrB))
    #print("hypoB:",hB)

    #C and not A,B,D
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 0,'B': 0,'C':1,'D':0} 
    y_copy = [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    theta_ovrC=  iterationr(y_temp,DataXX,ttheta,l)
    hC=sigmoid(DataXX.dot(theta_ovrC))
    #print("hypoC:",hC)

    #D and not A,B,C
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 0,'B': 0,'C':0,'D':1} 
    y_copy= [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    theta_ovrD=  iterationr(y_temp,DataXX,ttheta,l)
    hD=sigmoid(DataXX.dot(theta_ovrD))
    #print("hypoD:",hD)

    concat_hypo  = np.hstack([hA, hB, hC,hD])

    #Return index of max values
    hypo_final = np.argmax(concat_hypo,axis =1)
    print("hypo final for lambda ",l,":",hypo_final)

    testDataXX = np.insert(test_X,0,1,axis=1)
    test_hA=sigmoid(testDataXX.dot(theta_ovrA))
    test_hB=sigmoid(testDataXX.dot(theta_ovrB))
    test_hC=sigmoid(testDataXX.dot(theta_ovrC))
    test_hD=sigmoid(testDataXX.dot(theta_ovrD))
    concat_hypo  = np.hstack([test_hA, test_hB, test_hC,test_hD])

    #Return index of max values
    y_pred = np.argmax(concat_hypo,axis =1)
    print("hypo final test for lambda ",l,":",y_pred)
    confu_mat(y_test, y_pred)
    precision(y_test, y_pred)
    recall(y_test, y_pred)
    print(classification_report(y_test, y_pred, target_names=['A', 'B', 'C','D']))
    print("Accuracy:",accuracy_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in multiply
  del sys.path[0]


hypo final for lambda  0.0001 : [2 2 1 0 0 2 2 2 2 0 0 2 0 2 0 2 2 0 2 1 1 1 1 0 1 0 0 0 2 2 0 2 0 0 1 2 1
 2 2 0 2 2 2 0 2 0 1 2 2 1 2 2 0 2 2 1 0 0 2 0 2 0 0 2 2 0 2 2 2 0 1 0 0 1]
hypo final test for lambda  0.0001 : [2 1 2 0 1 2 2 2 1 1 2 2 2 0 1 1 0 1 2 2 0 1 2 0 1 0 0 0 2 2 1 2 2 2 2 1 2
 0]
[[5 2 0 0]
 [4 8 8 0]
 [0 1 8 0]
 [0 0 2 0]]
             precision    recall  f1-score   support

          A       0.56      0.71      0.63         7
          B       0.73      0.40      0.52        20
          C       0.44      0.89      0.59         9
          D       0.00      0.00      0.00         2

avg / total       0.59      0.55      0.53        38

Accuracy: 0.5526315789473685


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


hypo final for lambda  0.001 : [2 2 1 0 0 2 2 2 2 0 0 2 0 2 0 2 2 0 2 1 1 1 1 0 1 0 0 0 2 2 0 2 0 0 1 2 1
 2 2 0 2 2 1 0 2 0 1 2 2 1 2 1 0 2 2 1 0 0 2 0 2 0 0 2 2 0 2 2 2 0 0 0 0 1]
hypo final test for lambda  0.001 : [2 1 2 0 1 2 1 2 1 1 2 2 2 0 1 1 0 0 2 2 0 1 2 0 1 0 0 0 2 2 1 2 2 2 2 1 2
 0]
[[5 2 0 0]
 [5 7 8 0]
 [0 1 8 0]
 [0 1 1 0]]
             precision    recall  f1-score   support

          A       0.50      0.71      0.59         7
          B       0.64      0.35      0.45        20
          C       0.47      0.89      0.62         9
          D       0.00      0.00      0.00         2

avg / total       0.54      0.53      0.49        38

Accuracy: 0.5263157894736842
hypo final for lambda  0.01 : [2 2 1 0 0 2 2 2 2 0 0 2 0 1 0 2 1 0 2 1 1 0 0 0 0 0 0 0 2 2 0 2 0 0 1 2 0
 1 1 0 2 1 1 0 2 0 1 2 2 1 2 1 0 2 2 1 0 0 2 0 2 0 0 2 2 0 1 2 2 0 0 0 0 1]
hypo final test for lambda  0.01 : [2 0 2 0 0 2 1 2 0 1 1 2 2 0 1 1 0 0 1 2 0 0 2 0 1 0 0 0 1 1 1 2 1 2 1 1 2
 0]
[[7 0 0 0]
 [

In [19]:

confu_mat(y_test, y_pred)
precision(y_test, y_pred)
recall(y_test, y_pred)
#f1_score(y_test, y_pred)

[[ 0  7  0  0]
 [ 0 20  0  0]
 [ 0  9  0  0]
 [ 0  2  0  0]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Question 4c metrics

In [20]:
print(classification_report(y_test, y_pred, target_names=['A', 'B', 'C','D']))
print("Accuracy:",accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          A       0.00      0.00      0.00         7
          B       0.53      1.00      0.69        20
          C       0.00      0.00      0.00         9
          D       0.00      0.00      0.00         2

avg / total       0.28      0.53      0.36        38

Accuracy: 0.5263157894736842


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
